## Data Processing Pipeline for Soccer Player Embeddings

This notebook implements a comprehensive data processing pipeline for creating player embeddings from the European Soccer Database. The pipeline includes data extraction, temporal weighting, and feature normalization to prepare player profiles for machine learning applications.

### Key Features:
- Temporal weighting to emphasize recent player performance
- Comprehensive attribute normalization
- Player profile aggregation across multiple time periods
- Embedding-ready data preparation

### 1. Library Imports
Setting up the required libraries for data processing, machine learning, and database operations.

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import _sqlite3 as db
import pickle, json

### 2. Database Connection and Table Exploration
Connecting to the European Soccer Database and exploring available tables to understand the data structure.

In [3]:
# function for tables extraction
def extract_tables(cursor):
    for row in cursor.execute("SELECT name FROM sqlite_master WHERE type='table';"):
        print(row)

# Extraction tables from European Soccer Database (https://www.kaggle.com/datasets/hugomathien/soccer?resource=download)
con_euro = db.connect('Data/EuroSoccer.sqlite')
cursor = con_euro.cursor()

print("Tables from European Soccer Database:\n")
extract_tables(cursor)
print('\n')

Tables from European Soccer Database:

('sqlite_sequence',)
('Player_Attributes',)
('Player',)
('Match',)
('League',)
('Country',)
('Team',)
('Team_Attributes',)




### 3. Data Loading and Integration
Loading player information and attributes from the database, then merging them to create a comprehensive dataset with temporal player data.

In [4]:
players = pd.read_sql_query("SELECT * FROM Player", con_euro)
player_attributes = pd.read_sql_query("SELECT * FROM Player_Attributes", con_euro)

players_with_attributes = players.merge(
    player_attributes,
    on='player_api_id',
    how='left',
    suffixes=('', '_attr')
)

players_with_attributes = players_with_attributes.sort_values(
    ['player_api_id', 'date']
).reset_index(drop=True)

### 4. Data Quality Assessment
Exploring the dataset structure, checking for missing values, and understanding the temporal distribution of player records.

In [5]:
print(players_with_attributes.columns.tolist())
print(len(players_with_attributes.columns.tolist()))

['id', 'player_api_id', 'player_name', 'player_fifa_api_id', 'birthday', 'height', 'weight', 'id_attr', 'player_fifa_api_id_attr', 'date', 'overall_rating', 'potential', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate', 'crossing', 'finishing', 'heading_accuracy', 'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy', 'long_passing', 'ball_control', 'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots', 'aggression', 'interceptions', 'positioning', 'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle', 'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes']
48


In [6]:
print(f"Number of notes: {players_with_attributes.shape}")
print(f"Number of unique players: {players_with_attributes['player_api_id'].nunique()}")

Number of notes: (183978, 48)
Number of unique players: 11060


### 5. Temporal Weighting Implementation
Applying exponential decay weighting to player attributes to emphasize more recent performance data. This approach ensures that recent player form is weighted more heavily than historical data.

In [7]:
# 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate' - преобразовать в числа если нужны

attributes = [
    'overall_rating', 'potential', 'crossing', 'finishing', 'heading_accuracy', 'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy', 'long_passing', 'ball_control', 'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots', 'aggression', 'interceptions', 'positioning', 'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle', 'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes'
]

print("Missed values in attributes:")
for attr in attributes:
    if attr in players_with_attributes.columns:
        missing = players_with_attributes[attr].isnull().sum()
        print(f"{attr}: {missing} ({missing/len(players_with_attributes)*100:.1f}%)")
print(f"\nMean of notes by payers: {len(players_with_attributes)/players_with_attributes['player_api_id'].nunique():.1f}")
print(f"Number of players with one note: {(players_with_attributes['player_api_id'].value_counts() == 1).sum()}")


def apply_temporal_weights_to_players(players_df, decay_factor=0.95):
    weighted_players = []
    for player_id, group in players_df.groupby('player_api_id'):
        if group['date'].isnull().all():
            continue
        # sort by data from old -> to new
        group = group.sort_values('date').reset_index(drop=True)
        # calculate of weights
        n_records = len(group)
        weights = np.array([decay_factor ** (n_records - i - 1) for i in range(n_records)])
        # apply weights to attributes
        weighted_group = group.copy()
        for attr in attributes:
            if attr in group.columns and not group[attr].isnull().all():
                weighted_group[f'{attr}_weighted'] = group[attr] * weights
        weighted_players.append(weighted_group)
    return pd.concat(weighted_players, ignore_index=True)

# apply weights
players_weighted = apply_temporal_weights_to_players(players_with_attributes)
print(f"Data with: {players_weighted.shape}")

Missed values in attributes:
overall_rating: 836 (0.5%)
potential: 836 (0.5%)
crossing: 836 (0.5%)
finishing: 836 (0.5%)
heading_accuracy: 836 (0.5%)
short_passing: 836 (0.5%)
volleys: 2713 (1.5%)
dribbling: 836 (0.5%)
curve: 2713 (1.5%)
free_kick_accuracy: 836 (0.5%)
long_passing: 836 (0.5%)
ball_control: 836 (0.5%)
acceleration: 836 (0.5%)
sprint_speed: 836 (0.5%)
agility: 2713 (1.5%)
reactions: 836 (0.5%)
balance: 2713 (1.5%)
shot_power: 836 (0.5%)
jumping: 2713 (1.5%)
stamina: 836 (0.5%)
strength: 836 (0.5%)
long_shots: 836 (0.5%)
aggression: 836 (0.5%)
interceptions: 836 (0.5%)
positioning: 836 (0.5%)
vision: 2713 (1.5%)
penalties: 836 (0.5%)
marking: 836 (0.5%)
standing_tackle: 836 (0.5%)
sliding_tackle: 2713 (1.5%)
gk_diving: 836 (0.5%)
gk_handling: 836 (0.5%)
gk_kicking: 836 (0.5%)
gk_positioning: 836 (0.5%)
gk_reflexes: 836 (0.5%)

Mean of notes by payers: 16.6
Number of players with one note: 0
Data with: (183978, 83)


### 6. Player Profile Generation
Creating aggregated player profiles by computing weighted averages of all attributes for each player. This step consolidates multiple temporal records into a single representative profile.

In [8]:
def create_player_profiles(weighted_players_df):
    player_profiles = {}
    for player_id, group in weighted_players_df.groupby('player_api_id'):
        if group.empty:
            continue
        profile = {'player_api_id': player_id}
        # General data (just for checking)
        latest_record = group.iloc[-1]
        profile.update({
            'player_name': latest_record['player_name'],
            'height': latest_record['height'],
            'weight': latest_record['weight'],
            'preferred_foot': latest_record['preferred_foot'],
            'attacking_work_rate': latest_record['attacking_work_rate'],
            'defensive_work_rate': latest_record['defensive_work_rate']
        })
        # weighed number attributes
        weights = np.array([0.95 ** (len(group) - i - 1) for i in range(len(group))])
        for attr in attributes:
            if attr in group.columns and not group[attr].isnull().all():
                valid_data = group[attr].dropna()
                if len(valid_data) > 0:
                    # take according data
                    valid_weights = weights[-len(valid_data):]
                    profile[attr] = np.average(valid_data, weights=valid_weights)
        player_profiles[player_id] = profile
    return player_profiles

player_profiles = create_player_profiles(players_weighted)
print(f"Number of profiles: {len(player_profiles)}")


Number of profiles: 11060


### 7. Embedding Data Preparation
Preparing the final dataset for embedding generation by:
- Normalizing all numerical features using StandardScaler
- Creating player ID mappings for efficient indexing
- Saving the processed data for downstream applications

In [9]:
def prepare_embeddings_data(player_profiles):
    profiles_df = pd.DataFrame(list(player_profiles.values()))

    # mapping: player_api_id -> sequential_id
    unique_players = list(player_profiles.keys())
    player_id_to_idx = {pid: idx for idx, pid in enumerate(unique_players)}
    idx_to_player_id = {idx: pid for pid, idx in player_id_to_idx.items()}

    # normalization of data
    numeric_features = [attr for attr in attributes if attr in profiles_df.columns]

    scaler = StandardScaler()
    profiles_df[numeric_features] = scaler.fit_transform(
        profiles_df[numeric_features].fillna(profiles_df[numeric_features].mean())
    )

    return {
        'profiles_df': profiles_df,
        'player_id_to_idx': player_id_to_idx,
        'idx_to_player_id': idx_to_player_id,
        'scaler': scaler,
        'numeric_features': numeric_features
    }

embeddings_data = prepare_embeddings_data(player_profiles)
scaler = prepare_embeddings_data(player_profiles)['scaler']
print(f"Prepared for embedding: {len(embeddings_data['player_id_to_idx'])} players")

with open('embeddings_data.pkl', 'wb') as f:
    pickle.dump(embeddings_data, f)

print("embeddings saved as embeddings_data.pkl")

Prepared for embedding: 11060 players
embeddings saved as embeddings_data.pkl
